In [25]:
import os
import json
import numpy as np
import pandas as pd 

In [26]:
Home = os.getcwd()
DataFolder = os.path.join(Home,'Data')



In [27]:
MyDataFiles = [os.path.join(DataFolder,i) for i in os.listdir(DataFolder) if i.endswith('csv')]
lst = []
for MyFile in MyDataFiles:
    lst.append(pd.read_csv(MyFile))
    os.remove(MyFile)
df = pd.concat(lst)
df_cols = list(df.columns)
df = df.drop(columns=[i for i in df_cols if i.startswith('Unnamed:')])
df.to_csv(os.path.join(DataFolder,'MyDataSet.csv'))


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from niapy.algorithms import basic, modified

In [29]:
def Select_NIS_model(val:int=1,n_estimators:int=100):
    model = RandomForestClassifier()
    param_grid = {'n_estimators':[n_estimators]}
    if val == 1:
        return NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm='ba'), 'Bat Algorithm'
    elif val == 2:
        return NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm='hba'), 'Hybrid Bat Algorithm'
    elif val == 3:
        return NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm='fa'), 'Firefly Algorithm'
    elif val == 4:
        return NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm='hsaba'), 'Hybrid Self Adaptive Bat Algorithm'
    else:
        return NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm='gwo'), 'Grey Wolf Optimizer'
    '''
    ba, Bat Algorithm (population_size=40, loudness=1.0, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)

    hba, Hybrid Bat Algorithm (differential_weight=0.50, crossover_probability=0.90, strategy=cross_best1)

    fa, Firefly Algorithm ( population_size=20, alpha=1, beta0=1, gamma=0.01, theta=0.97)

    hsaba, Hybrid Self Adaptive Bat Algorithm (population_size=100, starting_loudness=0.5, epsilon=0.001, alpha=1.0, pulse_rate=0.5,min_frequency=0.0, max_frequency=2.0)
    
    gwo, Grey Wolf Optimizer
    '''

In [32]:
#Specific Value
def NIA_BAT():
    lst = []
    for My_population_size in range(10,51,5):
        for My_loudness in np.arange(0.5,1.1,.1):
            for My_pulse_rate in np.arange(0.5,1.1,.1):
                for My_alpha in np.arange(0.3,My_loudness-0.1,.1):
                    for My_gamma in np.arange(0.3,My_pulse_rate-0.1,.1):
                        for My_min_frequency in range(1,7,1):
                            for My_max_frequency in range(My_min_frequency+1,My_min_frequency+3,1):

                                Algo = basic.ba.BatAlgorithm()
                                #Vary, for NIA
                                Algo.set_parameters(
                                    population_size = My_population_size,
                                    loudness = My_loudness,
                                    pulse_rate = My_pulse_rate,
                                    alpha = My_alpha,
                                    gamma = My_gamma,
                                    min_frequency = My_min_frequency,
                                    max_frequency = My_max_frequency
                                )
                                lst.append(Algo)
    return lst
    '''default
    population_size=20,  range(10,51,5)
    loudness=1.0,       np.arange(0.5,1.1,.1)
    pulse_rate=1.0,    np.arange(0.5,1.1,.1)
    alpha=0.97,         np.arange(0.3,loudness-0.1,.1)
    gamma=0.1,          np.arange(0.3,pulse_rate-0.1,.1)
    min_frequency=0.0,   range(1,7,1)
    max_frequency=2.0,   min_frequency+1, +2
    '''
    #10*6*6*6*2*5*5

In [33]:
def Define_Model():
        model = RandomForestClassifier()
        param_grid = {}
        return model, param_grid
#Range
'''default
n_estimators=100, 
criterion='gini', 
max_depth=None, 
min_samples_split=2, 
min_samples_leaf=1, 
min_weight_fraction_leaf=0.0, 
max_features='sqrt', 
max_leaf_nodes=None, 
min_impurity_decrease=0.0, 
bootstrap=True, 
oob_score=False, 
n_jobs=None, 
random_state=None, 
verbose=0, 
warm_start=False, 
class_weight=None, 
ccp_alpha=0.0, 
max_samples=None)
'''


"default\nn_estimators=100, \ncriterion='gini', \nmax_depth=None, \nmin_samples_split=2, \nmin_samples_leaf=1, \nmin_weight_fraction_leaf=0.0, \nmax_features='sqrt', \nmax_leaf_nodes=None, \nmin_impurity_decrease=0.0, \nbootstrap=True, \noob_score=False, \nn_jobs=None, \nrandom_state=None, \nverbose=0, \nwarm_start=False, \nclass_weight=None, \nccp_alpha=0.0, \nmax_samples=None)\n"

In [34]:
df.head()

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,21,1,1,1
1,20,1,1,1
2,21,1,1,1
3,77,0,1,1
4,72,0,1,1


In [35]:
df_cols = list(df.columns)
y = df[df_cols[-1]]
X = df[df_cols[:-1]]


In [38]:
model, param_grid = Define_Model()
Algo_lst = NIA_BAT()
print(len(Algo_lst))
Best_Score = 0
Best_Para = None 
Bat_Dict = dict()
for i in range(len(Algo_lst)):
    Curr_Candidate = NatureInspiredSearchCV(estimator=model, param_grid=param_grid, algorithm=Algo_lst[i])
    Curr_Candidate.fit(X,y)
    Curr_Para = Algo_lst[i].get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    Bat_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para

with open('Kids Bat Dict.json','w') as kid:
    json.dump(Bat_Dict,kid)

print(Best_Score)
print(Best_Para)


90828
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization fi

KeyboardInterrupt: 

In [ ]:
def Model_try(DataFileName:str,model_val:int):
    df = pd.read_csv(DataFileName)
    df_cols = list(df.columns)
    y = df[df_cols[0]]
    X = df[df_cols[1:]]
    #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=False,random_state=1412)
    model, NIA_Name = Select_NIS_model(model_val)
    model.fit(X,y)
    print(f'For {NIA_Name} we get a score of {model.best_score_}.')
    #df = pd.DataFrame(model.cv_results_)

    #df.to_csv(f"{NIA_Name}_results.csv")
    return NIA_Name, model.best_score_


In [ ]:
def NIA_Apply(Algo, model, param_grid):
        #Fixed
    final = NatureInspiredSearchCV(
        estimator=model, 
        algorithm=Algo,
        param_grid=param_grid, 
        max_stagnating_gen=10,
        max_n_gen=100
        )
'''deafult
estimator, 
param_grid, 
algorithm='hba', 
population_size=50, 
max_n_gen=100, 
runs=3,
max_stagnating_gen=20, 
random_state=None, 
scoring=None, 
refit=True, 
verbose=0,
pre_dispatch='2*n_jobs', 
error_score=np.nan, 
return_train_score=True
             '''

In [ ]:
def Algo_Selector(val:int=1):
    if val == 1:
        print('Bat')
    elif val == 2:
        print('Hybrid Bat')
    elif val == 3:
        print('Self Adapting Hybrid Bat')
    else:
        print('Fire Fly')

In [ ]:
((10*6*6*6)+24+24)

2208

In [ ]:
def Model_try(DataFileName:str,model_val:int):
    df = pd.read_csv(DataFileName)
    df_cols = list(df.columns)
    y = df[df_cols[0]]
    X = df[df_cols[1:]]
    #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=False,random_state=1412)
    model, NIA_Name = Select_NIS_model(model_val)
    model.fit(X,y)
    print(f'For {NIA_Name} we get a score of {model.best_score_}.')
    #df = pd.DataFrame(model.cv_results_)

    #df.to_csv(f"{NIA_Name}_results.csv")
    return NIA_Name, model.best_score_


In [ ]:
MyDict = dict()
My_prev_idx = 0
for M_type in range(5):
    print(M_type)
    for Para in range(50,501,50):
        print(Para)
        start_time = dt.now()
        Name, Score = Model_try(DataFile,M_type,Para)
        end_time = dt.now()
        #total,cpu,monitor,disk,base, time_taken, end_idx
        total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)
        TempDict = {'Total Energy (J)' : total, 'Total CPU  Energy (J)' : cpu, 'Total Monitor Energy (J)' : monitor, 'Total Disk Energy (J)' : disk, 'Base Energy (J)' : base, 'Time (Sec)' : time_taken, 'Score' : Score*100, 'n_estimators' : Para}
        if Name in MyDict:
            MyDict[Name].append(TempDict)
        else:
            MyDict[Name] = [TempDict]
        with open('Kaito Kids Dairy.json','w') as kid:
            json.dump(MyDict,kid)


In [ ]:
#Specific Value
def NIA_Hybrid_Bat():
    Algo = modified.HybridBatAlgorithm()
    #Vary, for NIA
    Algo.set_parameters(
        
    )
    Algo.set_parameters(
        population_size = population_size,
        loudness = loudness,
        pulse_rate = pulse_rate,
        alpha = alpha,
        gamma = gamma,
        min_frequency = min_frequency,
        max_frequency = max_frequency
    )
    return Algo
    '''default
    differential_weight=0.5, 
    crossover_probability=0.9, 
    strategy=<function cross_best1 at 0x000001B81D4F64C0>
    '''
    #10*6*6*6 +24+24
